In [45]:
import sys

import numpy as np
import qiskit

from src.state_circuit_generator import StateCircuitGenerator, MHSTreeGeneratorExhaustive, MergingStatesGenerator, MHSTreeGeneratorHeuristic, QiskitDefaultGenerator
from src.utilities.general import make_dict
from src.utilities.qiskit_utilities import remove_leading_cx_gates
from src.utilities.validation import execute_circuit, get_state_vector, get_fidelity

sys.executable

'/home/teague/Projects/python-envs/qwalks-env/bin/python3.11'

In [48]:
def check_fidelity(circuit: qiskit.QuantumCircuit, target_state: dict[str, complex], fidelity_tol: float = 1e-8):
    output_state_vector = execute_circuit(circuit)
    target_state_vector = get_state_vector(target_state)
    fidelity = get_fidelity(output_state_vector[:len(target_state_vector)], target_state_vector)
    assert abs(1 - fidelity) < fidelity_tol, f"Failed to prepare the state. Fidelity: {fidelity}"
    return True

In [49]:
def prepare_walk_circ(
    target_state: dict[str, complex],
    circuit_generator: StateCircuitGenerator,
    basis_gates: list[str],
    optimization_level: int,
    ensure_fidelity: bool = False,
    fidelity_tol: float = 1e-8
) -> qiskit.QuantumCircuit:
    circuit = circuit_generator.generate_circuit(target_state)
    circuit_transpiled = qiskit.transpile(circuit, **make_dict(basis_gates, optimization_level))
    circuit_transpiled = remove_leading_cx_gates(circuit_transpiled)

    if ensure_fidelity:
        check_fidelity(circuit_transpiled, target_state)

    return circuit_transpiled

In [60]:
def generate_state(num_terms):
    num_qubits = (num_terms).bit_length()
    coeffs = np.random.uniform(0, 1, size=num_terms)
    coeffs = np.sqrt(coeffs / np.sum(coeffs)).tolist()
    bitstrings = [bin(i)[2:].zfill(num_qubits)[::-1] for i in range(2**num_qubits)] # Little endian because Qiskit
    return {state: amplitude for state, amplitude in zip(bitstrings[:len(coeffs)], coeffs)}

In [61]:
state = generate_state(10)
state

{'0000': 0.40406397879121436,
 '1000': 0.3176112071852002,
 '0100': 0.39887596769939404,
 '1100': 0.30403578247092955,
 '0010': 0.24373303527038465,
 '1010': 0.40326859935391973,
 '0110': 0.17777603484599938,
 '1110': 0.39160384831611245,
 '0001': 0.24537207019530505,
 '1001': 0.1308393150488911}

In [104]:
generators = [
    MHSTreeGeneratorExhaustive(change_basis=True, multiedge=True),
    MergingStatesGenerator(),
    MHSTreeGeneratorHeuristic(multiedge=False),
    QiskitDefaultGenerator()
]
prepare_circ = prepare_walk_circ(
    state,
    generators[0],
    basis_gates=["rx", "ry", "rz", "h", "cx"],
    optimization_level=3,
    ensure_fidelity=True
)

prepare_circ.draw(fold=-1)

┌───────┐                   ┌───┐┌────────┐┌──────────┐       ┌─────────┐  ┌────────────┐                   ┌─────────┐                                                                                                                                                                                                                                                                                                                                                                         ┌───────┐                                                                                                end 
q_0: ──┤ Rx(π) ├───────────────────┤ X ├┤ Rz(-π) ├┤ Ry(-π/2) ├──■────┤ Rz(π/2) ├──┤ Ry(2.8639) ├────────────────■──┤ Ry(π/2) ├──────────────────────────────────■────────────────────■───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■────────────────────■────┤ Rx(π) ├─────────────────────────────────────────────────────────────────────────────────────────────────░──
       └───────┘                   └─┬─┘└────────┘└──────────┘  │    └─────────┘  └────────────┘                │  └─────────┘               ┌───┐              │                    │                             ┌───┐┌─────────┐  ┌─────────┐          ┌─────────┐  ┌────────────┐                   ┌─────────┐  ┌──────────┐                                                          ┌───┐┌─────────┐  ┌───────┐                        ┌──────────┐    ┌───────┐      │                    │    └───────┘                                                                                                 ░  
q_1: ────────────────────────────────┼──────────────────────────┼───────────────────────────────────────────────┼────────────────────────────┤ X ├──────────────┼────────────────────┼─────────────────────────────┤ X ├┤ Rz(π/2) ├──┤ Ry(π/2) ├─────■────┤ Rz(π/2) ├──┤ Ry(2.7224) ├────────────────■──┤ Ry(π/2) ├──┤ Rz(-π/2) ├──────────────────────────────────────────────────────────┤ X ├┤ Rz(π/2) ├──┤ Ry(π) ├────────────────────■───┤ Rz(-π/2) ├────┤ Ry(π) ├──────┼────────────────────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────────░──
      ┌──────────┐ ┌──────────────┐  │                          │                                               │                            └─┬─┘              │                    │                             └─┬─┘└──┬───┬──┘  └─────────┘     │    └─────────┘  └────────────┘                │  └─────────┘  └──────────┘                                                          └─┬─┘└──┬───┬──┘ ┌┴───────┴┐     ┌───────┐     │   └──────────┘    └───────┘      │                    │                                    ┌────────────┐                                 ┌─────────┐ ┌─────────┐    ░  
q_2: ─┤ Rz(-π/2) ├─┤ Ry(0.097579) ├──■──────────────────────────┼───────────────────────────────────────────────┼──────────────────────────────┼────────────────┼────────────────────┼───────────────────────────────┼─────┤ X ├─────────────────────┼───────────────────────────────────────────────┼──────────────────────────────■─────────────────────■──────────────────────────────────┼─────┤ X ├────┤ Ry(π/2) ├─────┤ Rz(π) ├─────┼──────────────────────────────────┼────────────────────┼────────────────────────────────■───┤ Ry(1.5857) ├──────────────────────────────■──┤ Rz(π/2) ├─┤ Ry(π/2) ├────░──
     ┌┴──────────┴┐└──────────────┘                           ┌─┴─┐┌─────────────┐┌────────────┐┌────────────┐┌─┴─┐┌──────────┐┌────────────┐  │  ┌──────────┐┌─┴─┐┌──────────────┐┌─┴─┐┌─────────────┐┌──────────┐  │     └─┬─┘   ┌──────────────┐┌─┴─┐┌─────────────┐┌────────────┐┌────────────┐┌─┴─┐┌─────────┐┌─────────────┐┌─┴─┐┌───────────────┐┌─┴─┐┌──────────────┐┌────────────┐  │     └─┬─┘   ┌┴─────────┴─┐┌──┴───────┴──┐┌

In [110]:
def construct_select_circ(num_ham_qubits: int, H): # Haven't decided best way to represent H yet
    num_ctrl_qubits = (len(H)).bit_length()
    ctrl_qubits = [i for i in range(num_ctrl_qubits)]
    bitstrings = [bin(i)[2:].zfill(num_ctrl_qubits)[::-1] for i in range(2**num_ctrl_qubits)] # Little endian because Qiskit
    circ = qiskit.QuantumCircuit(num_ctrl_qubits + num_ham_qubits)
    for ctrl_bitstring, pauli_term in zip(bitstrings[:len(H)], H):
        #print(ctrl_bitstring, pauli_term)
        pauli_str = "".join([t[0] for t in pauli_term]) # The Pauli is always the first character
        qubit_idx = [num_ctrl_qubits + int(t[1:]) for t in pauli_term] # The qubit index is always everything after, offset the indices to account for the other register
        #print(pauli_str, qubit_idx)
        ctrl_pauli_gate = qiskit.circuit.library.PauliGate(pauli_str).control(num_ctrl_qubits=num_ctrl_qubits, ctrl_state=ctrl_bitstring)
        circ.append(ctrl_pauli_gate, ctrl_qubits + qubit_idx)
    return circ

In [94]:
H = [
    ('Z0',),
    ('X1',),
    ('Y0',),
    ('Z1',),
    ('X0', 'Z1'),
    ('Y0', 'Y1'),
    ('Z0', 'Z1'),
    ('X0', 'X1'),
    ('Z0', 'Y1'),
    ('Y1',),
]
select_circ = construct_select_circ(2, H)
select_circ.draw(fold=-1)

0000 ('Z0',)
Z [4]
1000 ('X1',)
X [5]
0100 ('Y0',)
Y [4]
1100 ('Z1',)
Z [5]
0010 ('X0', 'Z1')
XZ [4, 5]
1010 ('Y0', 'Y1')
YY [4, 5]
0110 ('Z0', 'Z1')
ZZ [4, 5]
1110 ('X0', 'X1')
XX [4, 5]
0001 ('Z0', 'Y1')
ZY [4, 5]
1001 ('Y1',)
Y [5]


q_0: ─────o───────────o───────────o───────────o────────────o─────────────o─────────────o─────────────o─────────────■────────────■──────
          │           │           │           │            │             │             │             │             │            │      
q_1: ─────o───────────o───────────o───────────o────────────■─────────────■─────────────■─────────────■─────────────o────────────o──────
          │           │           │           │            │             │             │             │             │            │      
q_2: ─────o───────────o───────────■───────────■────────────o─────────────o─────────────■─────────────■─────────────o────────────o──────
          │           │           │           │            │             │             │             │             │            │      
q_3: ─────o───────────■───────────o───────────■────────────o─────────────■─────────────o─────────────■─────────────o────────────■──────
     ┌────┴─────┐     │      ┌────┴─────┐     │      ┌─────┴──────┐┌─────┴──────┐┌─────┴──────┐┌─────┴──────┐┌─────┴──────┐     │      
q_4: ┤ Pauli(Z) ├─────┼──────┤ Pauli(Y) ├─────┼──────┤0           ├┤0           ├┤0           ├┤0           ├┤0           ├─────┼──────
     └──────────┘┌────┴─────┐└──────────┘┌────┴─────┐│  Pauli(XZ) ││  Pauli(YY) ││  Pauli(ZZ) ││  Pauli(XX) ││  Pauli(ZY) │┌────┴─────┐
q_5: ────────────┤ Pauli(X) ├────────────┤ Pauli(Z) ├┤1           ├┤1           ├┤1           ├┤1           ├┤1           ├┤ Pauli(Y) ├
                 └──────────┘            └──────────┘└────────────┘└────────────┘└────────────┘└────────────┘└────────────┘└──────────┘

In [124]:
def construct_qubitization_circ(num_qubits, H):
    generators = [
        MHSTreeGeneratorExhaustive(change_basis=True, multiedge=True),
        MergingStatesGenerator(),
        MHSTreeGeneratorHeuristic(multiedge=False),
        QiskitDefaultGenerator()
    ]
    
    prepare_circ = prepare_walk_circ(
        generate_state(len(H)),
        generators[0],
        basis_gates=["rx", "ry", "rz", "h", "cx"],
        optimization_level=3,
        ensure_fidelity=True
    )
    select_circ = construct_select_circ(num_qubits, H)

    qubitization_circ = qiskit.QuantumCircuit(num_qubits)
    # Put the PREPARE op on top of the circuit
    qubitization_circ.tensor(prepare_circ, inplace=True)
    prepare_op = qubitization_circ.copy()
    # Stick the SELECT op in
    qubitization_circ.compose(select_circ, qubits=qubitization_circ.qubits, inplace=True)
    # Finish with PREPARE^dag
    qubitization_circ.compose(prepare_op.inverse(), qubits=qubitization_circ.qubits, inplace=True)

    return qubitization_circ

In [125]:
H = [
    ('Z0',),
    ('X1',),
    ('Y0',),
    ('Z1',),
    ('X0', 'Z1'),
    ('Y0', 'Y1'),
    ('Z0', 'Z1'),
    ('X0', 'X1'),
    ('Z0', 'Y1'),
    ('Y1',),
]
qubitization_circ = construct_qubitization_circ(2, H)
qubitization_circ.draw(fold=-1)

┌───────┐                   ┌───┐┌─────────┐┌─────────┐       ┌─────────┐  ┌────────────┐                   ┌──────────┐ ┌───────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                end                                                                                                                                    end                                                                                                                                                                                                                                                                                                                                                                                                                          ┌──────────┐   ┌─────────┐                                                 ┌─────────────┐  ┌──────────┐                      ┌──────────┐  ┌──────────┐┌───┐    ┌────────┐               
q_0: ─┤ Rx(π) ├───────────────────┤ X ├┤ Rz(π/2) ├┤ Ry(π/2) ├──■────┤ Rz(π/2) ├──┤ Ry(2.6235) ├────────────────■──┤ Ry(-π/2) ├─┤ Rz(-5π/2) ├───────────────────────────────────────────────────■─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────░───────o───────────o───────────o───────────o────────────o─────────────o─────────────o─────────────o─────────────■────────────■────────░──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■────┤ Rz(5π/2) ├───┤ Ry(π/2) ├──────────────────────────────────────────────■──┤ Ry(-2.6235) ├──┤ Rz(-π/2) ├─────────────────■────┤ Ry(-π/2) ├──┤ Rz(-π/2) ├┤ X ├────┤ Rx(-π) ├───────────────
      └───────┘                   └─┬─┘└─────────┘└─────────┘  │    └─────────┘  └────────────┘                │  └──────────┘ └───────────┘ ┌───┐                                             │                                            ┌───┐┌──────────┐ ┌──────────┐        ┌─────────┐  ┌────────────┐                     ┌─────────┐    ┌──────────┐                                                 ┌───┐                                                                          │                                                                                                                               ░       │           │           │           │            │             │             │             │             │            │        ░                                                                                                                                  │                                                           ┌───┐  ┌─────────┐    ┌──────────┐                                                         ┌─────────────┐  ┌──────────┐                      ┌─────────┐  ┌─────────┐┌───┐                                             │    └──────────┘   └─────────┘                ┌───┐                         │  └─────────────┘  └──────────┘                 │    └──────────┘  └──────────┘└─┬─┘    └────────┘     

In [112]:
test = prepare_circ.copy()

test.draw(fold=-1)

┌───────┐                   ┌───┐┌────────┐┌──────────┐       ┌─────────┐  ┌────────────┐                   ┌─────────┐                                                                                                                                                                                                                                                                                                                                                                         ┌───────┐                                                                                                end 
q_0: ──┤ Rx(π) ├───────────────────┤ X ├┤ Rz(-π) ├┤ Ry(-π/2) ├──■────┤ Rz(π/2) ├──┤ Ry(2.8639) ├────────────────■──┤ Ry(π/2) ├──────────────────────────────────■────────────────────■───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■────────────────────■────┤ Rx(π) ├─────────────────────────────────────────────────────────────────────────────────────────────────░──
       └───────┘                   └─┬─┘└────────┘└──────────┘  │    └─────────┘  └────────────┘                │  └─────────┘               ┌───┐              │                    │                             ┌───┐┌─────────┐  ┌─────────┐          ┌─────────┐  ┌────────────┐                   ┌─────────┐  ┌──────────┐                                                          ┌───┐┌─────────┐  ┌───────┐                        ┌──────────┐    ┌───────┐      │                    │    └───────┘                                                                                                 ░  
q_1: ────────────────────────────────┼──────────────────────────┼───────────────────────────────────────────────┼────────────────────────────┤ X ├──────────────┼────────────────────┼─────────────────────────────┤ X ├┤ Rz(π/2) ├──┤ Ry(π/2) ├─────■────┤ Rz(π/2) ├──┤ Ry(2.7224) ├────────────────■──┤ Ry(π/2) ├──┤ Rz(-π/2) ├──────────────────────────────────────────────────────────┤ X ├┤ Rz(π/2) ├──┤ Ry(π) ├────────────────────■───┤ Rz(-π/2) ├────┤ Ry(π) ├──────┼────────────────────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────────░──
      ┌──────────┐ ┌──────────────┐  │                          │                                               │                            └─┬─┘              │                    │                             └─┬─┘└──┬───┬──┘  └─────────┘     │    └─────────┘  └────────────┘                │  └─────────┘  └──────────┘                                                          └─┬─┘└──┬───┬──┘ ┌┴───────┴┐     ┌───────┐     │   └──────────┘    └───────┘      │                    │                                    ┌────────────┐                                 ┌─────────┐ ┌─────────┐    ░  
q_2: ─┤ Rz(-π/2) ├─┤ Ry(0.097579) ├──■──────────────────────────┼───────────────────────────────────────────────┼──────────────────────────────┼────────────────┼────────────────────┼───────────────────────────────┼─────┤ X ├─────────────────────┼───────────────────────────────────────────────┼──────────────────────────────■─────────────────────■──────────────────────────────────┼─────┤ X ├────┤ Ry(π/2) ├─────┤ Rz(π) ├─────┼──────────────────────────────────┼────────────────────┼────────────────────────────────■───┤ Ry(1.5857) ├──────────────────────────────■──┤ Rz(π/2) ├─┤ Ry(π/2) ├────░──
     ┌┴──────────┴┐└──────────────┘                           ┌─┴─┐┌─────────────┐┌────────────┐┌────────────┐┌─┴─┐┌──────────┐┌────────────┐  │  ┌──────────┐┌─┴─┐┌──────────────┐┌─┴─┐┌─────────────┐┌──────────┐  │     └─┬─┘   ┌──────────────┐┌─┴─┐┌─────────────┐┌────────────┐┌────────────┐┌─┴─┐┌─────────┐┌─────────────┐┌─┴─┐┌───────────────┐┌─┴─┐┌──────────────┐┌────────────┐  │     └─┬─┘   ┌┴─────────┴─┐┌──┴───────┴──┐┌

In [113]:
ctrl_reg = qiskit.QuantumCircuit(2)
ctrl_reg.tensor(test, inplace=True)
ctrl_reg.draw(fold=-1)

┌───────┐                   ┌───┐┌────────┐┌──────────┐       ┌─────────┐  ┌────────────┐                   ┌─────────┐                                                                                                                                                                                                                                                                                                                                                                         ┌───────┐                                                                                                end 
q_0: ──┤ Rx(π) ├───────────────────┤ X ├┤ Rz(-π) ├┤ Ry(-π/2) ├──■────┤ Rz(π/2) ├──┤ Ry(2.8639) ├────────────────■──┤ Ry(π/2) ├──────────────────────────────────■────────────────────■───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■────────────────────■────┤ Rx(π) ├─────────────────────────────────────────────────────────────────────────────────────────────────░──
       └───────┘                   └─┬─┘└────────┘└──────────┘  │    └─────────┘  └────────────┘                │  └─────────┘               ┌───┐              │                    │                             ┌───┐┌─────────┐  ┌─────────┐          ┌─────────┐  ┌────────────┐                   ┌─────────┐  ┌──────────┐                                                          ┌───┐┌─────────┐  ┌───────┐                        ┌──────────┐    ┌───────┐      │                    │    └───────┘                                                                                                 ░  
q_1: ────────────────────────────────┼──────────────────────────┼───────────────────────────────────────────────┼────────────────────────────┤ X ├──────────────┼────────────────────┼─────────────────────────────┤ X ├┤ Rz(π/2) ├──┤ Ry(π/2) ├─────■────┤ Rz(π/2) ├──┤ Ry(2.7224) ├────────────────■──┤ Ry(π/2) ├──┤ Rz(-π/2) ├──────────────────────────────────────────────────────────┤ X ├┤ Rz(π/2) ├──┤ Ry(π) ├────────────────────■───┤ Rz(-π/2) ├────┤ Ry(π) ├──────┼────────────────────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────────░──
      ┌──────────┐ ┌──────────────┐  │                          │                                               │                            └─┬─┘              │                    │                             └─┬─┘└──┬───┬──┘  └─────────┘     │    └─────────┘  └────────────┘                │  └─────────┘  └──────────┘                                                          └─┬─┘└──┬───┬──┘ ┌┴───────┴┐     ┌───────┐     │   └──────────┘    └───────┘      │                    │                                    ┌────────────┐                                 ┌─────────┐ ┌─────────┐    ░  
q_2: ─┤ Rz(-π/2) ├─┤ Ry(0.097579) ├──■──────────────────────────┼───────────────────────────────────────────────┼──────────────────────────────┼────────────────┼────────────────────┼───────────────────────────────┼─────┤ X ├─────────────────────┼───────────────────────────────────────────────┼──────────────────────────────■─────────────────────■──────────────────────────────────┼─────┤ X ├────┤ Ry(π/2) ├─────┤ Rz(π) ├─────┼──────────────────────────────────┼────────────────────┼────────────────────────────────■───┤ Ry(1.5857) ├──────────────────────────────■──┤ Rz(π/2) ├─┤ Ry(π/2) ├────░──
     ┌┴──────────┴┐└──────────────┘                           ┌─┴─┐┌─────────────┐┌────────────┐┌────────────┐┌─┴─┐┌──────────┐┌────────────┐  │  ┌──────────┐┌─┴─┐┌──────────────┐┌─┴─┐┌─────────────┐┌──────────┐  │     └─┬─┘   ┌──────────────┐┌─┴─┐┌─────────────┐┌────────────┐┌────────────┐┌─┴─┐┌─────────┐┌─────────────┐┌─┴─┐┌───────────────┐┌─┴─┐┌──────────────┐┌────────────┐  │     └─┬─┘   ┌┴─────────┴─┐┌──┴───────┴──┐┌